In [2]:
import numpy as np

In [5]:
# finding the next state for the controls action possible
def control(state):
    x1 = state[0]
    y1 = state[1]
    controls = []
    dirc = [[0,0],[-1,0],[1,0],[0,-1],[0,1]]
    if(x1 ==1 and y1 == 1):
        controls = [1,1]
    elif(x1 == 2 and y1 == 2):
        controls = [2,2]
    else:
        for i in range(len(dirc)):
            z = np.array(dirc[i]) + np.array(state)
            x = z[0]
            y = z[1]
            c1  = z == np.array([1,1])
            c2 = z == np.array([2,2])
            e1 = c1.all()
            e2 = c2.all()
            if x>=0 and x<=3 :
                if y>=0 and y<=4:
                    if e1 == 0:
                        if e2 == 0:
                            controls.append(z)
    return(controls)

In [6]:
#takes the input as the next state and cost matrix and returns the value as list of as cost of next state
def next_v(mat,cnt):
    cont_val = []
    for i in range(len(cnt)):
        index = np.array(cnt[i])
        [x,y]= index
        val  = mat[x,y]
        cont_val.append(val)
    return(cont_val)

In [ ]:
## directions
def directionsfn(cost_matrix):
    dirc = [[0,0],[-1,0],[1,0],[0,-1],[0,1]]
    dirc_string = ['stay','up','down','left','right']
    a = np.chararray(cost_matrix.shape,itemsize=8)
    for i in range(cost_matrix.shape[0]):
        for j in range(cost_matrix.shape[1]) :
            dirc_c = []
            cost_w =[]
            if(i ==1 and j ==1) or (i == 2 and j == 2):
                a[i,j]='obs'
            else:
                for K in range(len(dirc)):
                    z = np.array(dirc[K])+np.array([i,j])
                    x = z[0]
                    y = z[1]
                    c1  = z == np.array([1,1])
                    c2 = z == np.array([2,2])
                    e1 = c1.all()
                    e2 = c2.all()
                    if x>=0 and x<=3 :
                        if y>=0 and y<=4:
                            if e1 == 0:
                                if e2 == 0:
                                    dirc_c.append(K)
                                    [dd,ff]= z
                                    cost_w.append(cost_matrix[dd,ff])

                ind  = cost_w.index(min(cost_w))
                a[i,j] = dirc_string[dirc_c[ind]]
    return(a)

In [7]:
# value iteration returns cost matrix and number of iterations
def value_iteration(ini,value_function,alpha):
    siz = ini.shape
    n = 0
    check = True
    while(check):
        Jnew = np.zeros(siz)
        n = n+1
        for i in range(siz[0]):
            for j in range(siz[1]):
                if (i ==1 and j == 1) or (i == 2 and j == 2):
                    continue            
                else:
                    l_jn_p1 = []
                    cont_cost = ini[i,j]
                    nex = control(np.array([i,j]))
                    value = next_v(value_function,nex)
                    for h in range(len(value)):
                        jn_plus1 = cont_cost + alpha * value[h]  
                        l_jn_p1.append(jn_plus1)
                    vall = min(l_jn_p1)
                    Jnew[i,j] = vall

        if (np.absolute(value_function - Jnew)< 1e-6).all():
            check = False
        else:
            value_function = Jnew
    return(Jnew, n)

In [60]:
ini = np.array([[0,0,1,0,-1],[0,0,1,0,10],[0,10,0,1,0],[1,0,0,10,-1]])
# initial value function
value_fn_ini = np.zeros(ini.shape)
alpha = 0.99
(cost_matrix,no_of_iterations) = value_iteration(ini,value_fn_ini,alpha)
# Printing cost matrix
print("The cost matrix is")
print(cost_matrix)
#Printing the no of iterations
print("No of iterations are")
print(no_of_iterations)
#directions
DIRC = directionsfn(cost_matrix)
print("The directions are")
print(DIRC)

The cost matrix is
[[-95.07940237 -96.03980137 -97.00990137 -98.99990137 -99.99990137]
 [-94.12860736   0.         -96.02980137 -98.00990137 -88.99990137]
 [-93.1873203  -82.25544611   0.         -97.00990137 -98.99990137]
 [-91.25544611 -90.34289066 -89.43946077 -88.99990137 -99.99990137]]
No of iterations are
1376
The directions are
[[b'right' b'right' b'right' b'right' b'stay']
 [b'up' b'obs' b'right' b'up' b'up']
 [b'up' b'left' b'obs' b'right' b'down']
 [b'up' b'left' b'left' b'right' b'stay']]


In [13]:
# Policy iteration returns no of iterations and cost matrix
def Policyiteration(A,G,ini,alpha):
    check = True
    siz = ini.shape
    y = 0
    while(check):
        y = y + 1
        A_new = np.zeros(A.shape)
        M = np.zeros(ini.shape)
        Jnew = np.linalg.solve(np.eye(A.shape[0])-alpha*A, G)
        [M[0,0],M[0,1],M[0,2],M[0,3],M[0,4],M[1,0]
        ,M[1,2],M[1,3],M[1,4],M[2,0],M[2,1],M[2,3],M[2,4],M[3,0],M[3,1],
        M[3,2],M[3,3],M[3,4]] = Jnew
        List = [[0,0],[0,1],[0,2],[0,3],[0,4],[1,0],[1,2],[1,3],[1,4],[2,0],
               [2,1],[2,3],[2,4],[3,0],[3,1],[3,2],[3,3],[3,4]]
        n = 0
        for i in range(siz[0]):
            for j in range(siz[1]):
                if (i ==1 and j == 1) or (i == 2 and j == 2):
                    continue            
                else:
                    l_jn_p1 = []
                    cont_cost = ini[i,j]
                    nex = control(np.array([i,j]))
                    value = next_v(M,nex)
                    for h in range(len(value)):
                        jn_plus1 = cont_cost + alpha * value[h]  
                        l_jn_p1.append(jn_plus1)
                    for_A_index = l_jn_p1.index(min(l_jn_p1))
                    A_index_cal = nex[for_A_index]
                    for jj in range(len(List)):
                        vl = np.array(List[jj])
                        c1 = vl == A_index_cal
                        e1 = c1.all()
                        if (e1 == 1):
                            A_new[n,jj]= 1
                n = n+1
        c2 = A == A_new
        e2 = c2.all()
        if (e2 == False):
            A = A_new
        else:
            check = False
    return(M,y)

In [64]:
#initial 
A = np.eye(18)
#continuous cost
G = np.array([[0,0,1,0,-1,0,1,0,10,0,10,1,0,1,0,0,10,-1]]).T
ini = np.array([[0,0,1,0,-1],[0,-10**5,1,0,10],[0,10,-10**5,1,0],[1,0,0,10,-1]])
alpha = 0.99
(cost_matrix_Pi,no_of_iteration_Pi) = Policyiteration(A,G,ini,alpha)
# Printing cost matrix
print("The cost matrix is")
print(cost_matrix_Pi)
#Printing the no of iterations
print("No of iterations are")
print(no_of_iteration_Pi)
DIRC_Pi = directionsfn(cost_matrix_Pi)
print("The directions are")
print(DIRC_Pi)

The cost matrix is
[[ -95.079501    -96.0399      -97.01        -99.         -100.        ]
 [ -94.12870599    0.          -96.0299      -98.01        -89.        ]
 [ -93.18741893  -82.25554474    0.          -97.01        -99.        ]
 [ -91.25554474  -90.34298929  -89.4395594   -89.         -100.        ]]
No of iterations are
10
The directions are
[[b'right' b'right' b'right' b'right' b'stay']
 [b'up' b'obs' b'right' b'up' b'up']
 [b'up' b'left' b'obs' b'right' b'down']
 [b'up' b'left' b'left' b'right' b'stay']]


5